In [1]:
from google.colab import drive

drive.mount('/content/drive')

chemin_zip_heavy_attacks = "/content/drive/My Drive/yvanolfotso/dataset/AttacksHeavy.zip"
chemin_zip_heavy_benign = "/content/drive/My Drive/yvanolfotso/dataset/BenignHeavy.zip"

chemin_zip_light_attacks = "/content/drive/My Drive/yvanolfotso/dataset/AttacksLight.zip"
chemin_zip_light_benign = "/content/drive/My Drive/yvanolfotso/dataset/BenignLight.zip"



Mounted at /content/drive


In [6]:
import zipfile
import os
import pandas as pd

import shutil

# Fonction pour extraire mes fichiers zip
def extraire_zip(chemin_zip):
    with zipfile.ZipFile(chemin_zip, 'r') as zip_ref:
        zip_ref.extractall("/content/extraction_temp")  # Extraction  dans mon répertoire temporaire

# Fonction pour charger les fichiers CSV d'un type spécifique (stateful ou stateless)
def charger_concatener_donnees(sous_dossier, prefixe):
    # Lister tous les fichiers CSV dans le sous-dossier
    fichiers_csv = [f for f in os.listdir(f"{sous_dossier}") if f.startswith(prefixe) and f.endswith('.csv')]
    # Lire chaque fichier CSV et le stocker dans une liste de DataFrames
    dataframes = [pd.read_csv(f"{sous_dossier}/{f}") for f in fichiers_csv]
    # Concaténer les DataFrames en un seul
    return pd.concat(dataframes, ignore_index=True)


# Extraire les fichiers zip
extraire_zip(chemin_zip_heavy_attacks)
extraire_zip(chemin_zip_heavy_benign)
extraire_zip(chemin_zip_light_attacks)
extraire_zip(chemin_zip_light_benign)

# Charger et concaténer les données de la même manière que vous l'avez fait auparavant
stateful_heavy_attack_data = charger_concatener_donnees("/content/extraction_temp/AttacksHeavy", "stateful")
stateful_heavy_benign_data = charger_concatener_donnees("/content/extraction_temp/BenignHeavy", "stateful")
stateless_heavy_attack_data = charger_concatener_donnees("/content/extraction_temp/AttacksHeavy", "stateless")
stateless_heavy_benign_data = charger_concatener_donnees("/content/extraction_temp/BenignHeavy", "stateless")

stateful_light_attack_data = charger_concatener_donnees("/content/extraction_temp/AttacksLight", "stateful")
stateful_light_benign_data = charger_concatener_donnees("/content/extraction_temp/BenignLight", "stateful")
stateless_light_attack_data = charger_concatener_donnees("/content/extraction_temp/AttacksLight", "stateless")
stateless_light_benign_data = charger_concatener_donnees("/content/extraction_temp/BenignLight", "stateless")

# Supprimer le répertoire temporaire après avoir terminé
if os.path.exists("/content/extraction_temp"):
    shutil.rmtree("/content/extraction_temp")


#### concatenation  sur axis = 0 ########

print(" cocncatenantion sur axis = 0 sur les y \n")
print(" Heavy attack")

heavy_attack = pd.concat([stateful_heavy_attack_data, stateless_heavy_attack_data], axis=0)
print(heavy_attack.shape)

#### j'ajoute la classe / label ######

heavy_attack['class'] = 'heavy_attacks'
print(heavy_attack.shape)

print(" \n")
print(" Heavy Bengnin")

heavy_bengin = pd.concat([stateful_heavy_benign_data, stateless_heavy_benign_data], axis=0)
print(heavy_bengin.shape)

#### j'ajoute la classe / label ######

heavy_bengin['class'] = 'heavy_bengnin'
print(heavy_bengin.shape)

print(" \n")
print(" Light attack")

light_attack = pd.concat([stateful_light_attack_data, stateless_light_attack_data], axis=0)
print(light_attack.shape)

#### j'ajoute la classe / label ######
light_attack['class'] = 'light_attacks'
print(light_attack.shape)


print(" \n")
print(" Light Bengnin")

light_bengin = pd.concat([stateful_light_benign_data, stateless_light_benign_data], axis=0)
print(light_bengin.shape)

#### j'ajoute la classe / label ######
light_bengin['class'] = 'light_bengnin'
print(light_bengin.shape)


final_data = pd.concat([heavy_attack, heavy_bengin,light_attack,light_bengin], axis=0, ignore_index=True)

# Vérifier les dimensions du jeu de données final
print(final_data.shape)


 cocncatenantion sur axis = 0 sur les y 

 Heavy attack
(323698, 42)
(323698, 43)
 

 Heavy Bengnin
(250710, 42)
(250710, 43)
 

 Light attack
(53978, 42)
(53978, 43)
 

 Light Bengnin
(82859, 42)
(82859, 43)
(711245, 43)


In [ ]:
import pandas as pd
from math import log
import matplotlib.pyplot as plt

def entropy(class_probabilities):
    """Calculates the entropy of a list of class probabilities."""
    return sum(-p * log(p, 2) for p in class_probabilities if p)

def information_gain(data, feature_name, classes):
    total_entropy = entropy(data['class'].value_counts(normalize=True))
    values = data[feature_name].unique()
    weighted_entropy = 0.0

    for v in values:
        subset_length = len(data[data[feature_name] == v])
        if subset_length == 0:
            continue  # Skip this value if subset length is zero to avoid division by zero
        subset_entropy = entropy(data[data[feature_name] == v]['class'].value_counts(normalize=True))
        weighted_entropy += (subset_length / len(data)) * subset_entropy

    return total_entropy - weighted_entropy

def calculate_information_gains(data, classes):
    information_gains = []
    total_entropy = entropy(data['class'].value_counts(normalize=True))

    for feature_name in data.columns[:-1]:  # Exclude the last column which is the target variable
        gain = information_gain(data, feature_name, classes)
        information_gains.append((feature_name, gain))

    return information_gains

def select_top_features(data, classes, top_n=10):
    information_gains = calculate_information_gains(data, classes)
    top_features = sorted(information_gains, key=lambda x: x[1], reverse=True)[:top_n]
    return top_features

# data = pd.read_csv("../data/BenignAndMaliciousDataset.csv")

data = final_data

classes = data['class'].unique()

# Calcul du gain d'information pour chaque caractéristique
# information_gains = calculate_information_gains(data, classes)

# Afficher les gains d'information pour chaque caractéristique
# for feature_name, gain in information_gains:
#     print("Information Gain for feature", feature_name, ":", gain)

# Sélectionner les 10 meilleures caractéristiques
top_features = select_top_features(data, classes, top_n=10)

# Afficher les noms des 10 meilleures caractéristiques et leur gain d'information
for feature_name, gain in top_features:
    print("Feature:", feature_name, "Information Gain:", gain)

# Tracer les 10 meilleures caractéristiques
plt.figure(figsize=(10, 6))
plt.bar([x[0] for x in top_features], [x[1] for x in top_features], color='skyblue')
plt.xlabel('Features')
plt.ylabel('Information Gain')
plt.title('Top 10 Features with Highest Information Gain')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()




In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from math import log

def entropy(class_probabilities):
    """Calculates the entropy of a list of class probabilities."""
    return sum(-p * log(p, 2) for p in class_probabilities if p)

def information_gain(data, feature_name, classes):
    total_entropy = entropy([len(data[data['class'] == c]) / len(data) for c in classes])
    values = data[feature_name].unique()
    weighted_entropy = 0.0

    for v in values:
        subset_length = len(data[data[feature_name] == v])
        if subset_length == 0:
            continue  # Skip this value if subset length is zero to avoid division by zero
        subset_entropy = entropy([len(data[(data[feature_name] == v) & (data['class'] == c)]) / subset_length for c in classes])
        weighted_entropy += (subset_length / len(data)) * subset_entropy

    return total_entropy - weighted_entropy

def calculate_information_gains(data, classes):
    information_gains = []

    for feature_name in data.columns[:-1]:  # Exclude the last column which is the target variable
        gain = information_gain(data, feature_name, classes)
        information_gains.append((feature_name, gain))

    return information_gains

def gain_ratio(data, feature_name, classes):
    gain = information_gain(data, feature_name, classes)
    iv = intrinsic_value(data, feature_name)
    if iv == 0:  # Avoid division by zero
        return 0
    return gain / iv

def intrinsic_value(data, feature_name):
    values = data[feature_name].unique()
    iv = sum(-(len(data[data[feature_name] == v]) / len(data)) * log(len(data[data[feature_name] == v]) / len(data), 2) for v in values if len(data[data[feature_name] == v]) > 0)
    return iv

def select_top_features_with_gain_ratio(data, classes, top_n=10):
    gain_ratios = [(feature_name, gain_ratio(data, feature_name, classes)) for feature_name in data.columns[:-1]]
    top_features = sorted(gain_ratios, key=lambda x: x[1], reverse=True)[:top_n]
    return top_features

def plot_top_features_gain_ratio(top_features, feature_names):
    top_feature_names = [feature_name for feature_name, _ in top_features]
    top_feature_ratios = [ratio for _, ratio in top_features]

    plt.figure(figsize=(10, 6))
    plt.bar(top_feature_names, top_feature_ratios, color='lightgreen')
    plt.xlabel('Features')
    plt.ylabel('Gain Ratio')
    plt.title('Top 10 Features by Gain Ratio')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# data = pd.read_csv("../data/BenignAndMaliciousDataset.csv")
data = final_data
classes = data['class'].unique()

# Sélectionner les 10 meilleures caractéristiques avec le ratio de gain
top_features_gain_ratio = select_top_features_with_gain_ratio(data, classes)

# Tracer les 10 meilleures caractéristiques avec le gain ratio
plot_top_features_gain_ratio(top_features_gain_ratio, data.columns[:-1])


In [ ]:
##### Dans ce cas plus la Gini est proche de Zeros plus plus le features est important cas il est pur (ie bon pour la sepration ) ##


import pandas as pd
import matplotlib.pyplot as plt

# Fonctions pour calculer le Gini Index
def gini_index(data, feature_name, classes):
    """Calculates the Gini index for a specific feature."""
    total_instances = len(data)
    values = data[feature_name].unique()
    gini_index_feature = 0.0

    for value in values:
        subset = data[data[feature_name] == value]
        probability = len(subset) / total_instances
        gini_index_feature += probability * gini(subset, classes)

    return gini_index_feature

def gini(data, classes):
    """Calculates the Gini index for a subset of data."""
    total_instances = len(data)
    gini_index_subset = 1.0

    for c in classes:
        num_instances_in_class = len(data[data['class'] == c])
        if num_instances_in_class == 0:
            continue
        probability = num_instances_in_class / total_instances
        gini_index_subset -= probability ** 2

    return gini_index_subset

def select_top_features_with_gini_index(data, classes, top_n=10):
    gini_indices = [(feature_name, gini_index(data, feature_name, classes)) for feature_name in data.columns[:-1]]
    top_features = sorted(gini_indices, key=lambda x: x[1])[:top_n]
    return top_features

# data = pd.read_csv("../data/BenignAndMaliciousDataset.csv")
data = final_data

classes = data['class'].unique()

# Sélectionner les 10 meilleures caractéristiques avec le Gini Index
top_features_gini_index = select_top_features_with_gini_index(data, classes)

# Afficher les noms et les indices de Gini des 10 meilleures caractéristiques
print("Top 10 Features with Gini Index:")
for feature_name, gini_value in top_features_gini_index:
    print("Feature:", feature_name, ", Gini Index:", gini_value)

# Tracer les 10 meilleures caractéristiques avec le Gini Index
def plot_top_features_gini_index(top_features, feature_names):
    top_feature_names = [feature_name for feature_name, _ in top_features]
    top_feature_gini = [gini_value for _, gini_value in top_features]

    plt.figure(figsize=(10, 6))
    colors = ['skyblue' if gini_value > 0 else 'red' for gini_value in top_feature_gini]  # Couleur spéciale pour les Gini Index de zéro
    plt.bar(top_feature_names, top_feature_gini, color=colors)
    plt.xlabel('Features')
    plt.ylabel('Gini Index')
    plt.title('Top Features by Gini Index')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Tracer les 10 meilleures caractéristiques avec le Gini Index
plot_top_features_gini_index(top_features_gini_index, data.columns[:-1])


In [ ]:
#### Cas ou le Labels n'es pas numerique il faut l'encoder d'abord ##########


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

def calculate_pearson_correlation(data):
    # Convertir les étiquettes de classe en valeurs numériques
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(data['class'])

    # Calculer le coefficient de corrélation de Pearson entre chaque caractéristique et la classe
    pearson_correlation = data.iloc[:, :-1].apply(lambda feature: np.abs(np.corrcoef(pd.to_numeric(feature, errors='coerce'), labels_encoded)[0, 1]))

    return pearson_correlation

def plot_top_features_pearson_correlation(pearson_correlation, feature_names):
    plt.figure(figsize=(10, 6))
    colors = ['orange' if correlation > 0 else 'red' for correlation in pearson_correlation]  # Couleur spéciale pour les coefficients de corrélation
    plt.bar(feature_names, pearson_correlation, color=colors)
    plt.xlabel('Features')
    plt.ylabel('Pearson Correlation')
    plt.title('Top Features by Pearson Correlation')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# data = pd.read_csv("../data/BenignAndMaliciousDataset.csv")

data = final_data

# Calculer les coefficients de corrélation de Pearson
pearson_correlation = calculate_pearson_correlation(data)

# Afficher les coefficients de corrélation de Pearson pour chaque caractéristique
print("Pearson Correlation for each feature:")
print(pearson_correlation)

# Afficher les caractéristiques avec leur coefficient de corrélation de Pearson
plot_top_features_pearson_correlation(pearson_correlation, data.columns[:-1])
